In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import data_analysis_tools as tools
import re
import random

## BASELINE

In [ ]:
OVERRIDE_OLD_FIXED = False
TEST = False

In [ ]:

base_path = f"{os.getcwd()}/../reports/"
baseline_paths = [
	f"{base_path}battery_1/",
]

neuroevolution_paths = [
	f"{base_path}battery_2/B1/",
	f"{base_path}battery_2/B1-path-grading-hack/",
	f"{base_path}battery_2/B1-path-grading-simple/",
	f"{base_path}battery_2/B2/",
	f"{base_path}battery_2/B3/",
	f"{base_path}battery_2/NB4/",
	f"{base_path}battery_2/NB4-path-grading/",

	# f"{base_path}neuroevolution/battery_3_0/",
]

neat_paths = [
	# f"{base_path}NEAT/battery_3_0/",
]
selected_paths = baseline_paths + neuroevolution_paths + neat_paths

In [ ]:
def prepare_data_fixed(path, selected_runs):
	for selected_run in selected_runs:
		this_path = f"{path}{selected_run}.data"
		if not os.path.exists(this_path + "_fixed") or OVERRIDE_OLD_FIXED:
			if not os.path.exists(this_path) or OVERRIDE_OLD_FIXED:
				#join data if necessary
				tools.join_data(this_path)
			tools.quote_gene(this_path)

def get_availiable_runs(filenames_list):
	selected_runs_set = set()
	for filename in filenames_list:
		found = re.findall("[0-9]+", filename)
		if len(found) > 0:
			selected_runs_set.add(found[0])
	return list(selected_runs_set)

def generate_run_dict(path):
	in_path_df_dict = dict()
	errors = list()
	
	for run in get_availiable_runs(os.listdir(path)):
		this_baseline_mode = None
		this_ann_layers = None
		print(path, run)
		run_dict = dict()
		run_dict["run_id"] = run
		run_dict["path"] = path
		with open(f"{path}{run}.conf", 'r') as f:
			for line in f.readlines():
				if tools.ANN_MODE_STRING in line:
					this_mode = line.replace(tools.ANN_MODE_STRING, "")
					run_dict["mode"] = this_mode
				elif tools.ANN_LAYERS_STRING in line:
					this_layers = line.replace(tools.ANN_LAYERS_STRING, "")
					run_dict["layers"] = this_layers

		run_dict["df"], load_errors = tools.load_dataframe(path, run)
		in_path_df_dict[run] = run_dict
		errors.extend(load_errors)
		if TEST == True:
			return in_path_df_dict, errors
	return in_path_df_dict, errors


## Do it

In [ ]:
errors = list()
run_dict_dict = dict()
for path in selected_paths:
	run_dict_dict[path], load_errors = generate_run_dict(path)
	errors.extend(load_errors)
	if TEST == True:
		break

# run count
run_count = 0
for index, path_dict in run_dict_dict.items():
	for _, run_dict in path_dict.items():
		run_count = run_count + 1
		errors.extend(tools.generate_run_report_from_dict(run_dict))

print("Path count: ", len(run_dict_dict.keys()))
print("Run count: ", run_count)
print("Errors:\n" + str(errors))